In [3]:
import pandas as pd
import numpy as np
from erddapy import ERDDAP
import re

In [4]:
# Let's define a tool that look through a server for datasets containing the listed standard variables
# Output is a table that list
# + server
# + dataset ID
# + distinct location (lat/long)
# + standard variables and their corresponding variable name
# + Earlier and latest time attached to a location
# + Shallowest and deepest depth attached to a location


def get_standard_variables_and_metadata(server_link, standard_variable_list):

    # Get access to the server and find datasets associated with standard_name variable listed
    e = ERDDAP(server=server_link,
               protocol='tabledap',
               response='csv')

    # Define Filter for which datasets to look into
    kw = {'standard_name': ','.join(standard_variable_list),
          'min_lon': -180.0, 'max_lon': 180.0,
          'min_lat': -90.0, 'max_lat': 90.0,
          'min_time': '', 'max_time': '',
          'cdm_data_type': ''}

    variable_to_groupby = [('latitude', 'degrees_north'), ('longitude', 'degrees_east')]

    # Get available datasets from that server
    search_url = e.get_search_url(response='csv', **kw)
    datasets = pd.read_csv(search_url)

    # Print results
    print(e.server)
    print(str(len(datasets))+" datasets contains "+', '.join(standard_variable_list))

    # Loop through different data sets and create a metadata dataFrame
    df = pd.DataFrame(columns=['Dataset ID'])

    for index, row in datasets.iterrows():
        # Get Info from dataset (mostly min/max lat/long)
        print(row['Dataset ID'])
        info_url = e.get_info_url(dataset_id=row['Dataset ID'], response='csv')
        info = pd.read_csv(info_url)
        attribute_table = info.set_index(['Row Type','Variable Name','Attribute Name']).transpose()['attribute']

        # Try to get the distinct lat/long and time and depth range for that dataset, if it fails rely on the
        # ERDDAP metadata
        try:
            # If dataset is spread out geographically find distinct locations (may not work well for trajectory data)
            latlong_url = e.get_download_url(dataset_id=row['Dataset ID'],
                                             protocol='tabledap',
                                             variables=['latitude', 'longitude', 'time'])

            # Get add to the url commands to get distinct values and ordered with min and max time for each lat/long
            distinctMinMaxTime_url = latlong_url+'&distinct()&orderByMinMax(%22latitude%2Clongitude%2Ctime%22)'

            # Get lat/long and min/max depth for this dataset
            data = pd.read_csv(distinctMinMaxTime_url, header=[0, 1])

            # Group data by latitude/longitude and get min max values
            data_reduced = data.groupby(by=variable_to_groupby).agg(['min', 'max']).reset_index()

            if info[(info['Variable Name'] == 'depth')].size > 0:
                latlongdepth_url = e.get_download_url(dataset_id=row['Dataset ID'],
                                                      protocol='tabledap',
                                                      variables=['latitude', 'longitude', 'depth'])

                # Get add to the url commands to get distinct values and ordered with min and max depth for
                # each lat/long
                distinctMinMaxDepth_url = latlongdepth_url + \
                                          '&distinct()&orderByMinMax(%22latitude%2Clongitude%2Cdepth%22)'

                # Get lat/long and min/max depth for this dataset
                data_depth = pd.read_csv(distinctMinMaxDepth_url, header=[0, 1])

                # Group depth data by lat/long and get min max values
                data_depth_reduced = data_depth.groupby(by=variable_to_groupby).agg(['min', 'max']).reset_index()

                # Merge depth values with time
                data_reduced = data_reduced.merge(data_depth_reduced, on=variable_to_groupby, how='left')

            # Merge multi index column names
            data_reduced.columns = data_reduced.columns.map(' '.join).str.strip(' ')

        except Exception as exception_error:

            print('Failed to read: ' + str(exception_error))
            # If there's only one location, it could get the range from metadata

            # Find lat/long range of this dataset, if it's point we don't need to look into it
            min_latitude = float(attribute_table['NC_GLOBAL', 'geospatial_lat_min'].Value)
            max_latitude = float(attribute_table['NC_GLOBAL', 'geospatial_lat_max'].Value)
            min_longitude = float(attribute_table['NC_GLOBAL', 'geospatial_lon_min'].Value)
            max_longitude = float(attribute_table['NC_GLOBAL', 'geospatial_lon_max'].Value)

            # If min/max lat/long are the same don't go in the dataset
            if (min_latitude == max_latitude) & (min_longitude == max_longitude):
                data_reduced = pd.DataFrame(columns=['Dataset ID'])
                data_reduced = {}
                data_reduced['latitude degrees_north'] = min_latitude
                data_reduced['longitude degrees_east'] = min_longitude

                if 'depth' in attribute_table.columns and 'actual_range' in attribute_table['depth'] and ('m' == attribute_table['depth', 'units']['Value']):

                        depth_range = np.array(str.split(attribute_table['depth', 'actual_range']['Value'], ',')).astype(np.float)
                        data_reduced['depth m min'] = depth_range[0]
                        data_reduced['depth m max'] = depth_range[1]

                # Convert to DataFrame
                data_reduced = pd.DataFrame(data_reduced, index=[0])
                print('Retrieved metadata')
            else:
                # Won't handle data with multiple location that it can't retrieve the data
                continue

        # Add Standard Name Variable Name to table info['Attribute Name'] == 'geospatial_lat_min'
        for var in standard_variable_list:
            data_reduced[var] = ','.join(e.get_var_by_attr(dataset_id=row['Dataset ID'],  standard_name=var))

        # Add cdm_data_type to table
        data_reduced['cdm_data_type'] = ','.join(info[info['Attribute Name'] == 'cdm_data_type']['Value'].values)

        # Add Dataset id to the table
        data_reduced['Dataset ID'] = row['Dataset ID']

        # Merge that dataset ID with previously downloaded data
        df = df.append(data_reduced)

    # Add server to dataFrame
    df['server'] = e.server

    # Save resulting dataframe to a CSV, file name is based on the server address
    file_name = re.sub('https*://', '', e.server)
    file_name = re.sub("[\./]", '_', file_name)
    file_name = 'Server_List_'+file_name+'.csv'

    print('Save result to '+file_name)
    df.to_csv(file_name)

    return df

In [5]:
# ERDDAP Links from CIOOS Pacific
cioos_pacific_servers = ['https://catalogue.hakai.org/erddap',
                        'http://dap.onc.uvic.ca/erddap',
                        'https://data.cioospacific.ca/erddap']

standard_variable_list_to_look_for = ['sea_water_practical_salinity', 'sea_water_temperature']

# Get metadata for each server and datasets
for server in cioos_pacific_servers:
    get_standard_variables_and_metadata(server, standard_variable_list_to_look_for)

https://catalogue.hakai.org/erddap
20 datasets contains sea_water_practical_salinity, sea_water_temperature
HakaiSewardBoL5min
HakaiKetchikanBoL5min
Failed to read: 404 Client Error:  for url: https://catalogue.hakai.org/erddap/tabledap/HakaiKetchikanBoL5min.csv?latitude,longitude,time
Retrieved metadata
HakaiKodiakBoL5min
HakaiCTDDFO2
HakaiCTDFZH01
HakaiCTDKC10
HakaiCTDPRUTH
Failed to read: 404 Client Error:  for url: https://catalogue.hakai.org/erddap/tabledap/HakaiCTDPRUTH.csv?latitude,longitude,time
Retrieved metadata
HakaiCTDQCS01
Failed to read: 404 Client Error:  for url: https://catalogue.hakai.org/erddap/tabledap/HakaiCTDQCS01.csv?latitude,longitude,time
Retrieved metadata
HakaiCTDQU20
Failed to read: 404 Client Error:  for url: https://catalogue.hakai.org/erddap/tabledap/HakaiCTDQU20.csv?latitude,longitude,time
Retrieved metadata
HakaiCTDQU29
Failed to read: 404 Client Error:  for url: https://catalogue.hakai.org/erddap/tabledap/HakaiCTDQU29.csv?latitude,longitude,time
Retrie

Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1189273.csv?latitude,longitude,time
Retrieved metadata
scalar_1192790
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1192790.csv?latitude,longitude,time
Retrieved metadata
scalar_1200625
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1200625.csv?latitude,longitude,time
Retrieved metadata
scalar_1194371
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1194371.csv?latitude,longitude,time
Retrieved metadata
currents_1194371
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/currents_1194371.csv?latitude,longitude,time
Retrieved metadata
scalar_117206
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erdd

Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/profile_117757.csv?latitude,longitude,time
Retrieved metadata
profile_118837
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/profile_118837.csv?latitude,longitude,time
Retrieved metadata
profile_1190485
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/profile_1190485.csv?latitude,longitude,time
Retrieved metadata
profile_1200170
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/profile_1200170.csv?latitude,longitude,time
Retrieved metadata
profile_118840
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/profile_118840.csv?latitude,longitude,time
Retrieved metadata
profile_1190483
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/er

Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1205219.csv?latitude,longitude,time
Retrieved metadata
scalar_1203032
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1203032.csv?latitude,longitude,time
Retrieved metadata
scalar_1189202
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1189202.csv?latitude,longitude,time
Retrieved metadata
profile_1190487
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/profile_1190487.csv?latitude,longitude,time
Retrieved metadata
scalar_1190474
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1190474.csv?latitude,longitude,time
Retrieved metadata
scalar_1192793
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erdda

Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/currents_1189466.csv?latitude,longitude,time
Retrieved metadata
scalar_1190641
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1190641.csv?latitude,longitude,time
Retrieved metadata
scalar_117406
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_117406.csv?latitude,longitude,time
Retrieved metadata
scalar_118131
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_118131.csv?latitude,longitude,time
Retrieved metadata
scalar_1189474
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1189474.csv?latitude,longitude,time
Retrieved metadata
scalar_1190647
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/ta

...

Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1203274.csv?latitude,longitude,time
Retrieved metadata
scalar_1205785
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1205785.csv?latitude,longitude,time
Retrieved metadata
scalar_1203269
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1203269.csv?latitude,longitude,time
Retrieved metadata
scalar_1205756
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1205756.csv?latitude,longitude,time
Retrieved metadata
scalar_1203270
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1203270.csv?latitude,longitude,time
Retrieved metadata
scalar_1205788
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/

Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1197996.csv?latitude,longitude,time
Retrieved metadata
scalar_1199494
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1199494.csv?latitude,longitude,time
Retrieved metadata
scalar_1201706
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1201706.csv?latitude,longitude,time
Retrieved metadata
scalar_1203094
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1203094.csv?latitude,longitude,time
Retrieved metadata
scalar_1202592
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1202592.csv?latitude,longitude,time
Retrieved metadata
currents_1202592
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erdda

Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_117541.csv?latitude,longitude,time
Retrieved metadata
scalar_118223
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_118223.csv?latitude,longitude,time
Retrieved metadata
scalar_117895
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_117895.csv?latitude,longitude,time
Retrieved metadata
scalar_118118
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_118118.csv?latitude,longitude,time
Retrieved metadata
scalar_118534
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_118534.csv?latitude,longitude,time
Retrieved metadata
scalar_118706
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/s

Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/currents_1201145.csv?latitude,longitude,time
Retrieved metadata
scalar_1206427
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1206427.csv?latitude,longitude,time
Retrieved metadata
currents_1206427
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/currents_1206427.csv?latitude,longitude,time
Retrieved metadata
scalar_22703
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_22703.csv?latitude,longitude,time
Retrieved metadata
scalar_1566
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1566.csv?latitude,longitude,time
Retrieved metadata
scalar_30141
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabled

Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1192544.csv?latitude,longitude,time
Retrieved metadata
scalar_1191108
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1191108.csv?latitude,longitude,time
Retrieved metadata
scalar_117343
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_117343.csv?latitude,longitude,time
Retrieved metadata
scalar_1202479
Failed to read: 500 Server Error: Internal Server Error for url: http://dap.onc.uvic.ca/erddap/tabledap/scalar_1202479.csv?latitude,longitude,time
Retrieved metadata
Save result to Server_List_dap_onc_uvic_ca_erddap.csv
https://data.cioospacific.ca/erddap
5 datasets contains sea_water_practical_salinity, sea_water_temperature
IOS_CTD_Profiles
IOS_ADCP_Moorings
IOS_CTD_Moorings
IOS_BOT_Profiles
ECCC_swob_marine
Save result to Server_List_data_cioospacific_